In [36]:
json_file = "../output/data/scrapy/pomelocare.com.json"

output_file_base = "../output/data/scrapy/pomelocare.com"

In [37]:
import json

with open(json_file, "r") as file:
    data = json.load(file)

total_chars = 0
for page in sorted(data, key=lambda p: p["url"]):
    truncated_page = page["markdown"][:4000]
    print(f"""
# {page["url"]}
{len(page["markdown"]):,} chars in markdown

{truncated_page}...
""")
    total_chars += len(truncated_page)

print(f"Total chars: {total_chars:,}")


# https://app.pomelocare.com/en/login?ajs_event=checkeligibility&ajs_prop_src=homepage
91 chars in markdown

# [](https://app.pomelocare.com/en/login?ajs_event=checkeligibility&ajs_prop_src=homepage)
...


# https://app.pomelocare.com/en/login?ajs_event=getstarted&ajs_prop_src=homepage
85 chars in markdown

# [](https://app.pomelocare.com/en/login?ajs_event=getstarted&ajs_prop_src=homepage)
...


# https://app.pomelocare.com/en/login?ajs_event=meetcareteam&ajs_prop_src=homepage
87 chars in markdown

# [](https://app.pomelocare.com/en/login?ajs_event=meetcareteam&ajs_prop_src=homepage)
...


# https://app.pomelocare.com/en/login?ajs_event=registertoday&ajs_prop_src=forpatients
91 chars in markdown

# [](https://app.pomelocare.com/en/login?ajs_event=registertoday&ajs_prop_src=forpatients)
...


# https://app.pomelocare.com/en/login?ajs_event=seeifeligible&ajs_prop_src=forpatients
91 chars in markdown

# [](https://app.pomelocare.com/en/login?ajs_event=seeifeligible&ajs_prop_src=forpatie

In [38]:
from core import init

init()

# Version 1: Basic summary

In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# Use the output for summarization

# Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create a ChatPromptTemplate with system guidance
prompt = ChatPromptTemplate([
    ("system", "You are a summarization assistant. Summarize the following markdown content."),
    ("user", "{markdowns}"),
])

# Extract the truncated markdowns from the data
markdowns = "\n\n".join(page["markdown"][:4000] for page in data)

# Create the LLMChain with the prompt template and the model
chain = prompt | llm

# Run the chain with the markdowns as input
summary = chain.invoke({"markdowns": markdowns})

print(summary.content)

# Write the summary to a markdown file
summary_file_path = f"{output_file_base}_summary.md"
with open(summary_file_path, "w") as summary_file:
    summary_file.write(summary.content)

**Pomelo Care Overview**

Pomelo Care is a virtual maternity care program that provides 24/7 access to a dedicated care team, including nurses, midwives, and other specialists, to support patients throughout their pregnancy journey. The service is designed to complement in-person care from OB-GYNs or midwives, offering personalized care plans to address various pregnancy-related issues, such as nausea and hypertension. Pomelo is available at no cost to members of participating health plans and employers.

**Key Features:**
- **24/7 Access:** Patients can reach their care team via text, phone, or video.
- **Personalized Care Plans:** Tailored support for managing pregnancy challenges.
- **Dedicated Care Teams:** Each patient is assigned a nurse care partner and matched with a team based on their specific needs.
- **Collaboration with In-Person Providers:** Pomelo works alongside existing healthcare providers to ensure comprehensive care.

**Recent Developments:**
- Pomelo has raised $46

# Version 2: Ask questions then summarize

In [40]:
# Define a list of questions about the company
questions = [
    "How does the company make money?",
    "Is the company more B2C or B2B?",
    "If B2B, what are the typical customers? Please include example customers if known.",
    "If B2C, what are the typical customer demographics?",
    "When was the company founded?",
    "How many employees does the company have?",
    "What is the company mission?",
    "What funding rounds has the company raised?",
    "What products does the company produce?",
    "Describe the scale of the company if possible, such as the number of customers, users, clients, or revenue.",
    "How are the company's products distributed or sold?",
    "How has the company changed over time?",
    "Does the company have any third-party certifications or awards?",

# - When was the company founded?
# - Approximately how many employees work at the company?
# - What products does the company produce? What services does the company offer?
# - How does the company make money? Who are their customers in general? Is it B2B, B2C? If B2B, include example customers.
# - Approximately how much revenue does the company generate annually?
# - Describe the scale of the company if possible, including the number of customers, users, or clients.
# - How are the company's products distributed or sold to users?
# - How has the company changed over time?
# - How do third parties describe the company, compared to how the company describes itself?
# - Acquisitions
# - Partnerships
# - Fundraising events
# - Opinions about the company
# - The scale of the company in terms of employee, active users, or revenue
# - New product developments
# - Information about any company executives including any relevant quotes
# - General information about the company
# - General information about the product
# - Any major changes in the company or product 

]

# Create a ChatPromptTemplate with system guidance and questions
question_prompt = ChatPromptTemplate(
    [
        ("system", "You are an assistant that answers questions about a company based on provided markdown content."),
        ("user", "Documents: \n\n{markdowns}"),
        ("user", "Questions to answer:\n" + "\n".join(questions)),
    ]
)

# Create the LLMChain with the question prompt template and the model
question_chain = question_prompt | llm

# Run the chain with the markdowns as input
answers = question_chain.invoke({"markdowns": markdowns})

# Print the answers
print(answers.content)

Here are the answers to your questions based on the provided content about Pomelo Care:

1. **How does the company make money?**
   Pomelo Care makes money by partnering with employers and health plans to offer its virtual maternity care services. The services are typically provided at no cost to patients who are members of participating health plans or employers.

2. **Is the company more B2C or B2B?**
   Pomelo Care operates primarily as a B2B (business-to-business) company, partnering with health plans and employers to provide services to their members.

3. **If B2B, what are the typical customers? Please include example customers if known.**
   Typical customers include health plans, employers, and Medicaid managed care organizations. Specific example customers mentioned include Mount Sinai Health System and Penn Medicine.

4. **If B2C, what are the typical customer demographics?**
   While the primary model is B2B, the end users (patients) are typically expecting mothers and famil

In [41]:
# Create a new ChatPromptTemplate with system guidance, including the answered questions
stage2_prompt = ChatPromptTemplate(
    [
        ("system", "You are a summarization assistant. Summarize the following markdown content, considering the provided answers to questions about the company."),
        ("user", "Documents: \n\n{markdowns}"),
        ("user", "Answers to questions:\n{answered_questions}"),
    ]
)

# Create the LLMChain with the new prompt template and the model
stage2_chain = stage2_prompt | llm

# Run the chain with the markdowns as input
stage2_summary = stage2_chain.invoke({"markdowns": markdowns, "answered_questions": answers.content})

print(stage2_summary.content)

# Write the stage2 summary to a markdown file
guided_summary_file_path = f"{output_file_base}_guided_summary.md"
with open(guided_summary_file_path, "w") as guided_summary_file:
    guided_summary_file.write(stage2_summary.content)

**Pomelo Care Overview**

Pomelo Care is a virtual maternity care provider founded in 2021, focused on improving maternal and newborn health outcomes through accessible, evidence-based care. The company operates primarily on a B2B model, partnering with health plans and employers to offer its services at no cost to eligible patients. Typical customers include Medicaid managed care organizations and employers like Mount Sinai Health System and Penn Medicine.

**Services Offered**
Pomelo provides a comprehensive virtual maternity care program that includes:
- Personalized care plans tailored to individual needs.
- Access to a dedicated care team comprising midwives, nurses, dietitians, and therapists.
- 24/7 support via text, phone, or video.

**Funding and Growth**
Pomelo has raised a total of $79 million through various funding rounds, including $33 million in Seed and Series A funding in 2023 and $46 million in Series B funding in 2024. The company has rapidly expanded its reach, now 

# Version 3: A more reasonable one-stage prompt

In [42]:
# Create a new ChatPromptTemplate with system guidance, including the questions
single_stage_prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a summarization assistant. Summarize the following markdown content, considering the following questions for guidance:\n"
            + "\n".join(questions),
        ),
        ("user", "Documents: \n\n{markdowns}"),
    ]
)

# Create the LLMChain with the new prompt template and the model
single_stage_chain = single_stage_prompt | llm

# Run the chain with the markdowns as input
single_stage_summary = single_stage_chain.invoke({"markdowns": markdowns})

print(single_stage_summary.content)

# Write the single-stage summary to a markdown file
single_stage_summary_file_path = f"{output_file_base}_single_stage_summary.md"
with open(single_stage_summary_file_path, "w") as single_stage_summary_file:
    single_stage_summary_file.write(single_stage_summary.content)

**Pomelo Care Summary**

- **Business Model**: Pomelo Care operates a virtual maternity care program that provides 24/7 personalized support to pregnant individuals and new parents. The service is offered at no cost to members of participating health plans or employers, making it a B2B model primarily targeting health plans and employers.

- **Target Customers**: Typical customers include health plans, employers, and Medicaid managed care organizations. Example partners include Mount Sinai Health System and Penn Medicine.

- **Founded**: Pomelo Care was founded in 2021.

- **Employees**: The company has a diverse team of clinicians, technologists, and executives, although the exact number of employees is not specified.

- **Mission**: Pomelo's mission is to improve maternal and newborn health outcomes by providing accessible, evidence-based care and addressing health inequities.

- **Funding Rounds**: Pomelo has raised a total of $79 million across multiple funding rounds, including a 